In [1]:
import pickle
import pandas as pd
from tqdm import tqdm

In [2]:
from UMLS import UMLS

In [3]:
umls = UMLS()

Loading Raw MRCONSO Lines


15229292it [00:51, 298058.60it/s]                                                                                                      


Creating mappings between concept IDs for easy access.


 55%|██████████████████████████████████████████████▋                                      | 8363207/15229292 [01:37<12:22, 9241.26it/s]

('A23137822', 'C1677279', 'Phylloscopus ricketti (organism)', '160071000009102', 'SNOMEDCT_VET', 'S', 'PF', 'ENG', 'L11342451')


...

 87%|████████████████████████████████████████████████████████████████████████▋           | 13182314/15229292 [02:37<04:06, 8288.48it/s]

('A26788584', 'C4113638', 'Milnesium sp. Ta46-01', '1511404', 'NCBI', 'P', 'PF', 'ENG', 'L13289158')


100%|███████████████████████████████████████████████████████████████████████████████████▌| 15157306/15229292 [03:01<00:15, 4721.15it/s]

('A31985697', 'C5347719', 'Grosmannia sp. MY-2019g', '2664516', 'NCBI', 'P', 'PF', 'ENG', 'L16476801')


100%|███████████████████████████████████████████████████████████████████████████████████| 15229292/15229292 [03:02<00:00, 83428.35it/s]


In [4]:
old_edges = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/umls2020AB_sapbert_edges.p','rb'))

In [5]:
old_edges_df = pd.DataFrame(old_edges)

In [6]:
old_edges[-2:]

[('MITCH', 'Herrick', 0), ('MITCH', 'Mila', 0)]

In [7]:
old_edges_df.groupby(2).count()

,0,1
2,,
0,73398038,73398179
1,1411834,1411834


In [8]:
with open('/data/Bodenreider_UMLS_DL/AAAI2022/UMLS_VERSIONS/2020AA-ACTIVE/NEGPOS1_WITHOUT_RAN/GENTEST_DS/2020AA-ACTIVE_ALL_MODEL_GENTEST_DS_TEST_DS.RRF','r') as f:
    
    lines = f.readlines()

In [9]:
lines[:10]

['0.34|A10415696|A26688809|1\n',
 '1.0|A30930715|A31248057|1\n',
 '0.2|A24181155|A29526244|1\n',
 '0.0|A2887200|A2873222|1\n',
 '1.0|A27068388|A18648350|1\n',
 '0.26|A19606779|A31074020|1\n',
 '0.5|A7797393|A1644567|1\n',
 '0.0|A18652928|A18560176|1\n',
 '0.0|A3160558|A19404565|1\n',
 '0.25|A25717689|A18657012|1\n']

In [10]:
lex_lm = []

for line in tqdm(lines):
    line = line.strip().split('|')
    
    lex_lm.append(tuple(line))

lexlm_df = pd.DataFrame(lex_lm)

100%|████████████████████████████████████████████████████████████████████████████████| 173035861/173035861 [03:32<00:00, 816087.35it/s]


In [11]:
lexlm_df

,0,1,2,3
0,0.34,A10415696,A26688809,1
1,1.0,A30930715,A31248057,1
2,0.2,A24181155,A29526244,1
3,0.0,A2887200,A2873222,1
4,1.0,A27068388,A18648350,1
...,...,...,...,...
173035856,0,A29014911,A18540132,0
173035857,0,A18103929,A4428515,0
173035858,0,A18103929,A19257752,0
173035859,0,A18103929,A14148740,0


In [12]:
lexlm_df.groupby(3).count()

,0,1,2
3,,,
0,167454653,167454653,167454653
1,5581208,5581208,5581208


In [13]:
translational_dev_set = pickle.load(open('/data/Bodenreider_UMLS_DL/Interns/Bernal/rw_uva/output/2/classifiers/1/aui_splits.p','rb'))

In [14]:
#Convert 2020AB development set into UBERT and LexLM Friendly formats

In [15]:
translational_dev_set = translational_dev_set['dev']

In [16]:
ubert_dev_set = []

for aui_h, aui_t, label in tqdm(translational_dev_set):
    
    ubert_dev_set.append((umls.aui2str[aui_h], umls.aui2str[aui_t], label))

100%|██████████████████████████████████████████████████████████████████████████████████████| 146255/146255 [00:00<00:00, 283513.79it/s]


In [17]:
pickle.dump(ubert_dev_set, open('../data/ubert_2020AB_dev_set.p','wb'))

In [18]:
lexlm_dev_set = []
lexlm_dev_set2 = []

for aui_h, aui_t, label in tqdm(translational_dev_set):
    
    lexlm_dev_set.append('0.0|{}|{}|{}'.format(label, aui_h, aui_t))
    lexlm_dev_set2.append('0.0|{}|{}|{}'.format(aui_h, aui_t, label))

100%|██████████████████████████████████████████████████████████████████████████████████████| 146255/146255 [00:00<00:00, 559931.95it/s]


In [19]:
lexlm_dev_set

['0.0|0|A31705996|A29264942',
 '0.0|0|A32287991|A1144104',
 '0.0|0|A32290281|A7759394',
 '0.0|0|A31918580|A17488742',
 '0.0|0|A32269634|A15268036',
 '0.0|0|A31687156|A29204823',
 '0.0|0|A31894895|A30506335',
 '0.0|0|A32043264|A20375469',
 '0.0|0|A31622747|A3073218',
 '0.0|0|A31889138|A2138874',
 '0.0|0|A31899857|A18692610',
 '0.0|0|A31804749|A23811603',
 '0.0|0|A32266145|A14170984',
 '0.0|0|A32037157|A28158972',
 '0.0|0|A32051789|A20339748',
 '0.0|0|A31825218|A6818265',
 '0.0|0|A31830457|A7592960',
 '0.0|0|A31694610|A8438586',
 '0.0|0|A31845562|A20693764',
 '0.0|0|A32333709|A7810368',
 '0.0|0|A31662713|A24042703',
 '0.0|0|A31829896|A16473006',
 '0.0|0|A32270674|A3399892',
 '0.0|1|A31816194|A11994996',
 '0.0|0|A31889138|A7114418',
 '0.0|0|A31726755|A18794412',
 '0.0|0|A32065740|A23681028',
 '0.0|0|A31634526|A7587964',
 '0.0|0|A31973092|A26871338',
 '0.0|0|A31659952|A15882900',
 '0.0|0|A31865461|A8587706',
 '0.0|0|A31956871|A28126484',
 '0.0|0|A31821086|A20733590',
 '0.0|1|A32084107|A283

In [20]:
with open('../data/lexlm_2020AB_dev_set.RRF','w') as f:
    for l in lexlm_dev_set:
        f.write(l+'\n')

In [21]:
with open('../data/lexlm_2020AB_dev_set_2.RRF','w') as f:
    for l in lexlm_dev_set2:
        f.write(l+'\n')